# Quantitative Value Strategy
"Value investing" means investing in the stocks that are cheapest relative to common measures of business value (like earnings or assets).

For this project, we're going to build an investing strategy that selects the 50 stocks with the best value metrics. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.

## Library Imports
The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [1]:
import numpy as np #The Numpy numerical computing library
import pandas as pd #The Pandas data science library
import requests #The requests library for HTTP requests in Python
import xlsxwriter #The XlsxWriter libarary for 
import math #The Python math module
from scipy import stats #The SciPy stats module

## Importing Our List of Stocks & API Token
As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the .csv file is still in your working directory and import it with the following command:

In [2]:
stocks = pd.read_csv('sp_500_stocks.csv')
from secrets import IEX_CLOUD_API_TOKEN


## Making Our First API Call
It's now time to make the first version of our value screener!

We'll start by building a simple value screener that ranks securities based on a single metric (the price-to-earnings ratio).

In [3]:
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
data

{'avgTotalVolume': 80307330,
 'calculationPrice': 'close',
 'change': -2.79,
 'changePercent': -0.01903,
 'close': 152.28,
 'closeSource': 'ilfaoifc',
 'closeTime': 1634050904569,
 'companyName': 'Apple Inc',
 'currency': 'USD',
 'delayedPrice': 149,
 'delayedPriceTime': 1638845761126,
 'extendedChange': -0.32,
 'extendedChangePercent': -0.00213,
 'extendedPrice': 146.43,
 'extendedPriceTime': 1659085195825,
 'high': 154.56,
 'highSource': 'uiieemcyn et5 pedadl1r ',
 'highTime': 1677329262326,
 'iexAskPrice': None,
 'iexAskSize': None,
 'iexBidPrice': None,
 'iexBidSize': None,
 'iexClose': 149.66,
 'iexCloseTime': 1686259251456,
 'iexLastUpdated': None,
 'iexMarketPercent': None,
 'iexOpen': 150.6,
 'iexOpenTime': 1696388525303,
 'iexRealtimePrice': None,
 'iexRealtimeSize': None,
 'iexVolume': None,
 'lastTradeTime': 1686639981542,
 'latestPrice': 151.69,
 'latestSource': 'Close',
 'latestTime': 'September 17, 2021',
 'latestUpdate': 1685749092901,
 'latestVolume': 135453900,
 'low':

## Parsing Our API Call
This API call has the metric we need - the price-to-earnings ratio.

Here is an example of how to parse the metric from our API call:

In [4]:
pe_ratio = data['peRatio']
pe_ratio

29.06

## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called chunks that we can use to divide our list of securities into groups of 100.

In [5]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

my_columns = ['Ticker', 'Price', 'Price-to-Earnings Ratio', 'Number of Shares to Buy']

Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [6]:
final_dataframe = pd.DataFrame(columns = my_columns)

for symbol_string in symbol_strings:
#     print(symbol_strings)
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        final_dataframe = final_dataframe.append(
                                        pd.Series([symbol, 
                                                   data[symbol]['quote']['latestPrice'],
                                                   data[symbol]['quote']['peRatio'],
                                                   'N/A'
                                                   ], 
                                                  index = my_columns), 
                                        ignore_index = True)
        
    
final_dataframe

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,A,178.27,55.29,N/A
1,AAL,20.17,-2.08,N/A
2,AAP,208.90,22.4,N/A
3,AAPL,147.01,28.69,N/A
4,ABBV,109.77,30.48,N/A
...,...,...,...,...
500,YUM,132.08,30.39,N/A
501,ZBH,152.13,33.46,N/A
502,ZBRA,573.77,40.52,N/A
503,ZION,58.57,5.91,N/A


## Removing Glamour Stocks

The opposite of a "value stock" is a "glamour stock". 

Since the goal of this strategy is to identify the 50 best value stocks from our universe, our next step is to remove glamour stocks from the DataFrame.

We'll sort the DataFrame by the stocks' price-to-earnings ratio, and drop all stocks outside the top 50.

In [7]:
final_dataframe.sort_values('Price-to-Earnings Ratio', inplace = True)
final_dataframe = final_dataframe[final_dataframe['Price-to-Earnings Ratio'] > 0]
final_dataframe = final_dataframe[:50]
final_dataframe.reset_index(inplace = True)
final_dataframe.drop('index', axis=1, inplace = True)

## Calculating the Number of Shares to Buy
We now need to calculate the number of shares we need to buy. 

To do this, we will use the `portfolio_input` function that we created in our momentum project.

I have included this function below.

In [8]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input("Enter the value of your portfolio:")

    try:
        val = float(portfolio_size)
    except ValueError:
        print("That's not a number! \n Try again:")
        portfolio_size = input("Enter the value of your portfolio:")

Use the `portfolio_input` function to accept a `portfolio_size` variable from the user of this script.

In [9]:
portfolio_input()

You can now use the global `portfolio_size` variable to calculate the number of shares that our strategy should purchase.

In [10]:
position_size = float(portfolio_size) / len(final_dataframe.index)
for i in range(0, len(final_dataframe['Ticker'])):
    final_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / final_dataframe['Price'][i])
final_dataframe

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,EBAY,75.80,4,2638
1,MPC,61.07,5.18,3274
2,PRU,107.24,5.89,1864
3,ZION,58.57,5.91,3414
4,GM,53.15,5.98,3762
5,BIO,823.36,6.06,242
6,COF,163.32,6.48,1224
7,AFL,54.41,6.62,3675
8,CINF,117.20,6.94,1706
9,CE,153.03,6.99,1306


## Building a Better (and More Realistic) Value Strategy
Every valuation metric has certain flaws.

For example, the price-to-earnings ratio doesn't work well with stocks with negative earnings.

Similarly, stocks that buyback their own shares are difficult to value using the price-to-book ratio.

Investors typically use a `composite` basket of valuation metrics to build robust quantitative value strategies. In this section, we will filter for stocks with the lowest percentiles on the following metrics:

* Price-to-earnings ratio
* Price-to-book ratio
* Price-to-sales ratio
* Enterprise Value divided by Earnings Before Interest, Taxes, Depreciation, and Amortization (EV/EBITDA)
* Enterprise Value divided by Gross Profit (EV/GP)

Some of these metrics aren't provided directly by the IEX Cloud API, and must be computed after pulling raw data. We'll start by calculating each data point from scratch.

In [11]:
symbol = 'AAPL'
batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=advanced-stats,quote&symbols={symbol}&token={IEX_CLOUD_API_TOKEN}'
data = requests.get(batch_api_call_url).json()

# P/E Ratio
pe_ratio = data[symbol]['quote']['peRatio']

# P/B Ratio
pb_ratio = data[symbol]['advanced-stats']['priceToBook']

#P/S Ratio
ps_ratio = data[symbol]['advanced-stats']['priceToSales']

# EV/EBITDA
enterprise_value = data[symbol]['advanced-stats']['enterpriseValue']
ebitda = data[symbol]['advanced-stats']['EBITDA']
ev_to_ebitda = enterprise_value/ebitda

# EV/GP
gross_profit = data[symbol]['advanced-stats']['grossProfit']
ev_to_gross_profit = enterprise_value/gross_profit

Let's move on to building our DataFrame. You'll notice that I use the abbreviation `rv` often. It stands for `robust value`, which is what we'll call this sophisticated strategy moving forward.

In [12]:
rv_columns = [
    'Ticker',
    'Price',
    'Number of Shares to Buy', 
    'Price-to-Earnings Ratio',
    'PE Percentile',
    'Price-to-Book Ratio',
    'PB Percentile',
    'Price-to-Sales Ratio',
    'PS Percentile',
    'EV/EBITDA',
    'EV/EBITDA Percentile',
    'EV/GP',
    'EV/GP Percentile',
    'RV Score'
]

rv_dataframe = pd.DataFrame(columns = rv_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote,advanced-stats&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        enterprise_value = data[symbol]['advanced-stats']['enterpriseValue']
        ebitda = data[symbol]['advanced-stats']['EBITDA']
        gross_profit = data[symbol]['advanced-stats']['grossProfit']
        
        try:
            ev_to_ebitda = enterprise_value/ebitda
        except TypeError:
            ev_to_ebitda = np.NaN
        
        try:
            ev_to_gross_profit = enterprise_value/gross_profit
        except TypeError:
            ev_to_gross_profit = np.NaN
            
        rv_dataframe = rv_dataframe.append(
            pd.Series([
                symbol,
                data[symbol]['quote']['latestPrice'],
                'N/A',
                data[symbol]['quote']['peRatio'],
                'N/A',
                data[symbol]['advanced-stats']['priceToBook'],
                'N/A',
                data[symbol]['advanced-stats']['priceToSales'],
                'N/A',
                ev_to_ebitda,
                'N/A',
                ev_to_gross_profit,
                'N/A',
                'N/A'
        ],
        index = rv_columns),
            ignore_index = True
        )

{'week52change': 0.290414, 'week52high': 169.51, 'week52low': 90.41, 'marketcap': 75861250162, 'employees': 10724, 'day200MovingAvg': 134.17, 'day50MovingAvg': 148.64, 'float': 479955020, 'avg10Volume': 1641586.3, 'avg30Volume': 1479579.2, 'ttmEPS': 3.499, 'ttmDividendRate': 0.77, 'companyName': 'Zoetis, Inc.', 'sharesOutstanding': 494883276, 'maxChangePercent': 3.3525, 'year5ChangePercent': 1.69, 'year2ChangePercent': 0.608, 'year1ChangePercent': 0.281583, 'ytdChangePercent': 0.189876, 'month6ChangePercent': 0.10241, 'month3ChangePercent': 0.297048, 'month1ChangePercent': 0.159224, 'day30ChangePercent': 0.165634, 'day5ChangePercent': -0.012817, 'nextDividendDate': None, 'dividendYield': 0.00489706569361443, 'nextEarningsDate': '2020-10-13', 'exDividendDate': '2020-07-01', 'peRatio': 49.1, 'beta': 0.9584403132945124, 'totalCash': 1974321215, 'currentDebt': 549171552, 'revenue': 6518431250, 'grossProfit': 4133660573, 'totalRevenue': 6300683466, 'EBITDA': 2502772553, 'revenuePerShare': 1

## Dealing With Missing Data in Our DataFrame

Our DataFrame contains some missing data because all of the metrics we require are not available through the API we're using. 

You can use pandas' `isnull` method to identify missing data:

In [13]:
rv_dataframe[rv_dataframe.isnull().any(axis=1)]

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
28,ALXN,189.100,N/A,59.53,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
40,AON,292.450,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
71,BRK.B,277.240,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
118,CTL,12.000,N/A,9.91,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
136,DISCK,26.920,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
165,ETFC,50.310,N/A,14.59,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
186,FLIR,58.340,N/A,33.18,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
190,FOX,36.700,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
204,GOOG,2954.020,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
325,MXIM,104.520,N/A,35.46,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A


Dealing with missing data is an important topic in data science.

There are two main approaches:

* Drop missing data from the data set (pandas' `dropna` method is useful here)
* Replace missing data with a new value (pandas' `fillna` method is useful here)

In this tutorial, we will replace missing data with the average non-`NaN` data point from that column. 

Here is the code to do this:

In [14]:
for column in ['Price-to-Earnings Ratio', 'Price-to-Book Ratio','Price-to-Sales Ratio',  'EV/EBITDA','EV/GP']:
    rv_dataframe[column].fillna(rv_dataframe[column].mean(), inplace = True)

Now, if we run the statement from earlier to print rows that contain missing data, nothing should be returned:

In [15]:

rv_dataframe[rv_dataframe.isnull().any(axis=1)]

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score


## Calculating Value Percentiles

We now need to calculate value score percentiles for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* Price-to-earnings ratio
* Price-to-book ratio
* Price-to-sales ratio
* EV/EBITDA
* EV/GP

Here's how we'll do this:

In [16]:
metrics = {
            'Price-to-Earnings Ratio': 'PE Percentile',
            'Price-to-Book Ratio':'PB Percentile',
            'Price-to-Sales Ratio': 'PS Percentile',
            'EV/EBITDA':'EV/EBITDA Percentile',
            'EV/GP':'EV/GP Percentile'
}

for row in rv_dataframe.index:
    for metric in metrics.keys():
        rv_dataframe.loc[row, metrics[metric]] = stats.percentileofscore(rv_dataframe[metric], rv_dataframe.loc[row, metric])/100

# Print each percentile score to make sure it was calculated properly
for metric in metrics.values():
    print(rv_dataframe[metric])

#Print the entire DataFrame    
rv_dataframe

0      0.869307
1      0.075248
2      0.439604
3       0.59802
4      0.590099
         ...   
500    0.609901
501    0.667327
502    0.780198
503    0.087129
504    0.837624
Name: PE Percentile, Length: 505, dtype: object
0      0.817822
1      0.049505
2      0.514851
3      0.962376
4      0.879208
         ...   
500    0.045545
501    0.336634
502    0.835644
503    0.123762
504    0.930693
Name: PB Percentile, Length: 505, dtype: object
0      0.843564
1      0.065347
2      0.143564
3      0.788119
4      0.538614
         ...   
500    0.706931
501    0.571287
502     0.70297
503    0.419802
504    0.906931
Name: PS Percentile, Length: 505, dtype: object
0      0.910891
1      0.025743
2      0.324752
3      0.732673
4      0.287129
         ...   
500    0.714851
501    0.687129
502    0.875248
503    0.051485
504    0.908911
Name: EV/EBITDA Percentile, Length: 505, dtype: object
0      0.883168
1      0.075248
2      0.136634
3      0.879208
4      0.485149
         ...   
5

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,179.69,N/A,54.56,0.869307,10.86,0.817822,8.7300,0.843564,35.983964,0.910891,17.191323,0.883168,N/A
1,AAL,20.00,N/A,-2.08,0.075248,-1.72,0.049505,0.7147,0.065347,-4.598108,0.025743,1.837448,0.075248,N/A
2,AAP,211.50,N/A,22.94,0.439604,3.95,0.514851,1.1900,0.143564,11.251891,0.324752,2.672046,0.136634,N/A
3,AAPL,146.32,N/A,29.70,0.59802,37.90,0.962376,6.9800,0.788119,22.644485,0.732673,17.015626,0.879208,N/A
4,ABBV,108.49,N/A,29.38,0.590099,15.45,0.879208,3.6900,0.538614,10.499616,0.287129,7.442171,0.485149,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,132.80,N/A,30.27,0.609901,-4.82,0.045545,6.0800,0.706931,21.619619,0.714851,10.522547,0.685149,N/A
501,ZBH,147.60,N/A,34.56,0.667327,2.54,0.336634,3.9700,0.571287,20.426598,0.687129,6.679319,0.417822,N/A
502,ZBRA,590.75,N/A,42.00,0.780198,11.78,0.835644,6.0500,0.70297,30.971773,0.875248,12.850690,0.766337,N/A
503,ZION,58.59,N/A,5.74,0.087129,1.26,0.123762,2.7400,0.419802,4.697150,0.051485,2.527473,0.124752,N/A


## Calculating the RV Score
We'll now calculate our RV Score (which stands for Robust Value), which is the value score that we'll use to filter for stocks in this investing strategy.

The RV Score will be the arithmetic mean of the 4 percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the mean function from Python's built-in statistics module.

In [17]:
from statistics import mean

for row in rv_dataframe.index:
    value_percentiles = []
    for metric in metrics.keys():
        value_percentiles.append(rv_dataframe.loc[row, metrics[metric]])
    rv_dataframe.loc[row, 'RV Score'] = mean(value_percentiles)
    
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,179.69,N/A,54.56,0.869307,10.86,0.817822,8.7300,0.843564,35.983964,0.910891,17.191323,0.883168,0.86495
1,AAL,20.00,N/A,-2.08,0.075248,-1.72,0.049505,0.7147,0.065347,-4.598108,0.025743,1.837448,0.075248,0.058218
2,AAP,211.50,N/A,22.94,0.439604,3.95,0.514851,1.1900,0.143564,11.251891,0.324752,2.672046,0.136634,0.311881
3,AAPL,146.32,N/A,29.70,0.59802,37.90,0.962376,6.9800,0.788119,22.644485,0.732673,17.015626,0.879208,0.792079
4,ABBV,108.49,N/A,29.38,0.590099,15.45,0.879208,3.6900,0.538614,10.499616,0.287129,7.442171,0.485149,0.55604
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,132.80,N/A,30.27,0.609901,-4.82,0.045545,6.0800,0.706931,21.619619,0.714851,10.522547,0.685149,0.552475
501,ZBH,147.60,N/A,34.56,0.667327,2.54,0.336634,3.9700,0.571287,20.426598,0.687129,6.679319,0.417822,0.53604
502,ZBRA,590.75,N/A,42.00,0.780198,11.78,0.835644,6.0500,0.70297,30.971773,0.875248,12.850690,0.766337,0.792079
503,ZION,58.59,N/A,5.74,0.087129,1.26,0.123762,2.7400,0.419802,4.697150,0.051485,2.527473,0.124752,0.161386


## Selecting the 50 Best Value Stocks¶

As before, we can identify the 50 best value stocks in our universe by sorting the DataFrame on the RV Score column and dropping all but the top 50 entries.

In [18]:
rv_dataframe.sort_values(by = 'RV Score', inplace = True)
rv_dataframe = rv_dataframe[:50]
rv_dataframe.reset_index(drop = True, inplace = True)

## Calculating the Number of Shares to Buy
We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a for loop to calculate the number of shares to buy for each stock in our investment universe.

In [19]:
portfolio_input()

In [20]:
position_size = float(portfolio_size) / len(rv_dataframe.index)
for i in range(0, len(rv_dataframe['Ticker'])-1):
    rv_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / rv_dataframe['Price'][i])
rv_dataframe

C:\Users\Dell\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,UNM,25.12,79617,7.36,0.10495,0.4733,0.051485,0.3790,0.023762,3.292489,0.037624,0.379615,0.009901,0.045545
1,PRU,104.29,19177,5.67,0.085149,0.6407,0.055446,0.6378,0.057426,4.485095,0.045545,0.610921,0.013861,0.051485
2,AAL,20.00,100000,-2.08,0.075248,-1.7200,0.049505,0.7147,0.065347,-4.598108,0.025743,1.837448,0.075248,0.058218
3,L,53.84,37147,9.50,0.138614,0.7917,0.067327,0.9429,0.09505,5.147785,0.059406,0.906281,0.025743,0.077228
4,RE,263.20,7598,7.68,0.110891,1.0099,0.085149,0.9379,0.093069,6.692948,0.10099,0.872721,0.023762,0.082772
5,GM,53.30,37523,6.02,0.091089,1.4500,0.154455,0.5468,0.045545,2.307154,0.033663,2.081902,0.10099,0.085149
6,BA,219.15,9126,-14.27,0.047525,-7.7000,0.043564,2.0900,0.328713,-30.337875,0.007921,-206.840213,0.00396,0.086337
7,LNC,70.53,28356,9.60,0.143564,0.6071,0.053465,0.6592,0.061386,7.565980,0.158416,0.665733,0.015842,0.086535
8,HPQ,27.80,71942,8.76,0.128713,-8.4800,0.037624,0.5160,0.037624,6.318681,0.091089,2.768072,0.140594,0.087129
9,MCK,217.95,9176,-7.43,0.057426,-63.1200,0.013861,0.1340,0.007921,8.491402,0.207921,2.995296,0.152475,0.087921


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

In [21]:
writer = pd.ExcelWriter('value_strategy.xlsx', engine='xlsxwriter')
rv_dataframe.to_excel(writer, sheet_name='Value Strategy', index = False)

## Creating the Formats We'll Need For Our .xlsx File
You'll recall from our first project that formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase
* Float formats with 1 decimal for each valuation metric

Since we already built some formats in past sections of this course, I've included them below for you. Run this code cell before proceeding.

In [22]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

float_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [23]:

column_formats = {
                    'A': ['Ticker', string_template],
                    'B': ['Price', dollar_template],
                    'C': ['Number of Shares to Buy', integer_template],
                    'D': ['Price-to-Earnings Ratio', float_template],
                    'E': ['PE Percentile', percent_template],
                    'F': ['Price-to-Book Ratio', float_template],
                    'G': ['PB Percentile',percent_template],
                    'H': ['Price-to-Sales Ratio', float_template],
                    'I': ['PS Percentile', percent_template],
                    'J': ['EV/EBITDA', float_template],
                    'K': ['EV/EBITDA Percentile', percent_template],
                    'L': ['EV/GP', float_template],
                    'M': ['EV/GP Percentile', percent_template],
                    'N': ['RV Score', percent_template]
                 }

for column in column_formats.keys():
    writer.sheets['Value Strategy'].set_column(f'{column}:{column}', 25, column_formats[column][1])
    writer.sheets['Value Strategy'].write(f'{column}1', column_formats[column][0], column_formats[column][1])

## Saving Our Excel Output
As before, saving our Excel output is very easy:

In [24]:
writer.save()